# Defining A Custom Privacy Mechanisms

In this notebook we demonstrate how to extend the accountant to support custom privacy mechanisms.
For the sake of the tutorial we keep things simple by using the well known Laplace mechanism.

Assume we have a privacy mechanisms with a privacy curve $\delta(P || Q )$ where $P = \text{Lap}(\mu,1) $ and $ Q = \text{Lap}(0,1)$.
By Proposition B.2 in [[1](https://arxiv.org/pdf/2106.02848.pdf)], the privacy random variables $X$ and $Y$ are given by $ X = | Z | - | Z - \mu | $ and $ Y = | Z - \mu | - | Z | $ where $ Z \sim \text{Lap}(0,1)$.

More concretely, the CDF of $X$ is given by
$$
F_X(t) =
  \begin{cases}
    1 + \frac{1}{4} \exp( -t - \mu) \left ( -3 + \exp(2t) - 2t - 2\mu \right )    & \quad \text{if } t+\mu >0 \text{ and } t < 0 \\
    1 - \frac{1}{2} \exp( -t - \mu) \left ( 1 +  \mu \right )    & \quad \text{if } t \geq 0 \\
    \frac{1}{2} \exp(t) \cosh(\mu)                           & \quad \text{otherwise}
  \end{cases} \; .
$$

## Implementing the Privacy Random Variable

Given the derivation of $F_X$, we can now implement the corresponding `LaplaceMechanism` class which is a subclass of `PrivacyRandomVariable`.
The interface is lightweight and only the function `cdf` for the CDF is required to be overridden.

Optionally, an implementation for the Renyi divergences can be provided (see Definition 3 in [[2](https://ieeexplore.ieee.org/abstract/document/8049725)]).
This allows to automatically compute the required domain size.
If this is not provided the `eps_max` argument in the accountant needs to be set.


In [ ]:
import numpy as np
from prv_accountant import PrivacyRandomVariable

class LaplaceMechanism(PrivacyRandomVariable):
    def __init__(self, mu: float) -> None:
        self.mu = mu
        assert self.mu > 0

    def cdf(self, t):
        return np.where(np.logical_and(t < 0, t + self.mu > 0),
            1 + 1/4*np.exp(-t-self.mu)*(-3+np.exp(2*t)-2*t-2*self.mu),
            np.where(t > 0,
                1 - 1/2*np.exp(-t-self.mu)*(1+self.mu),
                1/2*np.exp(t)*np.cosh(self.mu)
            )
        )

    def rdp(self, alpha):
        return 1/(alpha-1) * np.log(
            alpha/(2*alpha-1) * np.exp((alpha-1)*self.mu) + (alpha-1)/(2*alpha-1) * np.exp(-alpha*self.mu)
        )

# Using the PRV class for the Laplace mechanism with the PRV accountant
From here on, we can use the PRV class just like any other provided PRVs such as the `PoissonSubsampledGaussianMechanism`.

In [ ]:
from prv_accountant import PRVAccountantHomogeneous
accountant = PRVAccountantHomogeneous(prv=LaplaceMechanism(0.01), eps_error=0.1, delta_error=1e-10, max_compositions=10_000)
accountant.compute_epsilon(delta=1e-6, num_compositions=1_000)

## References

[1] Sivakanth Gopi, Yin Tat Lee, Lukas Wutschitz. Numerical Composition of Differential Privacy. 35th Conference on Neural Information Processing Systems (NeurIPS 2021), Sydney, Australia.

[2] Ilya Mironov. Renyi Differential Privacy. 2017. IEEE 30th Computer Security Foundations Symposium